In [1]:
from mydbutils import make_connection, do_query_return_all
import csv
from pandas import DataFrame

In [2]:
conn_cars = make_connection(config_file = 'config-cars.ini')
cursor_cars = conn_cars.cursor()
conn_cars_analysis = make_connection(config_file = 'config-cars_analysis.ini')
cursor_cars_analysis = conn_cars_analysis.cursor()

In [3]:
def display_table(table, order_by=''):
    sql = f"SELECT * FROM {table}"
    
    if order_by != '':
        sql = sql + " ORDER BY " + order_by
    
    cursor_cars_analysis.execute(sql)

    columns = cursor_cars_analysis.description
    column_names = [column_info[0] for column_info in columns]

    df = DataFrame(cursor_cars_analysis.fetchall())
    df.columns = column_names
    
    return df

In [11]:
sql = ( """
        INSERT INTO cars_analysis.dm_car(car_id, 
                           car_name, 
                           car_make, 
                           car_type)
            SELECT car.car_id, car.car_name, car_make.make, car_type.type_name
            FROM cars.car, cars.car_make, cars.car_type LIMIT 10
        """
      )
cursor_cars.execute(sql)
conn_cars.commit()

display_table('dm_car', 'car_id')

,car_key,car_id,car_name,car_make,car_type
0,46,100,Vitara,Saturn,Hatchback
1,47,100,Vitara,Saturn,SUV
2,48,100,Vitara,Saturn,MUV
3,49,100,Vitara,Saturn,Sedan
4,50,100,Vitara,Saturn,Hybrid


In [12]:
sql = ( """
        INSERT INTO cars_analysis.dm_dealer(dealer_id, 
                              dealer_name, 
                              age, 
                              gender)
            SELECT DISTINCT dealer_id, dealer_name, age, gender
            FROM cars.dealer
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('dm_dealer', 'dealer_id')

ProgrammingError: 1146 (42S02): Table 'cars_analysis.dm_dealer' doesn't exist

In [ ]:
sql = ( """
        INSERT INTO cars_analysis.dm_region(reg_id, 
                              city)
            SELECT DISTINCT reg_id, city
            FROM cars.region
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('dm_region', 'reg_id')

In [ ]:
sql = ( """
        INSERT INTO cars_analysis.dm_calender(month, 
                           year)
            SELECT DISTINCT month, year
            FROM cars.car_sale
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('dm_calender', 'month')

In [ ]:
sql = ( """
        INSERT INTO cars_analysis.dm_customer(customer_id, 
                              customer_name, 
                              age, 
                              gender)
            SELECT DISTINCT customer_id, customer_name, age, gender
            FROM cars.customer
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('dm_customer', 'customer_id')

### insert into fact table

In [ ]:
sql = ( """
        INSERT INTO cars_analysis.fact_purchase(dealer_key, 
                                                car_key, 
                                                reg_key, 
                                                cal_key,
                                                sale_units)
            SELECT DISTINCT dealer_key, 
                            car_key, 
                            reg_key, 
                            cal_key,
                            sale_units.
            FROM cars_analysis.dm_dealer,cars_analysis.dm_car,
                 cars_analysis.dm_region,cars_analysis.dm_calender, 
                 cars.car_sale
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('fact_purchase', 'fact_purchase_key')

In [ ]:
sql = ( """
        INSERT INTO cars_analysis.fact_customer(customer_key,
                                                dealer_key, 
                                                car_key, 
                                                reg_key, 
                                                cal_key,
                                                sale_price)
            SELECT DISTINCT customer_key,
                            dealer_key, 
                            car_key, 
                            reg_key, 
                            cal_key,
                            sale_price
            FROM cars_analysis.dm_customer,cars_analysis.dm_dealer,
                 cars_analysis.dm_car,cars_analysis.dm_region,
                 cars_analysis.dm_calender, cars.price
        """
      )

cursor_cars.execute(sql)
conn_cars.commit()

display_table('fact_customer', 'fact_customer_key')

In [ ]:
cursor_cars.close()
conn_cars.close()
cursor_cars_analysis.close()
conn_cars_analysis.close()